In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [3]:
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/DATASET_annotations/instances_val2017_5labels.json"
!wget -nc "https://cleanlab-public.s3.amazonaws.com/ObjectDetectionBenchmarking/DATASET_annotations/instances_train2017_5labels.json"

File 'instances_val2017_5labels.json' already there; not retrieving.



Before you begin training on a custom dataset, be sure to review the COCO dataset guidelines for formatting your data, which can be found on their [website](https://cocodataset.org/#format-data)

To use a custom dataset named "my_dataset" with detectron2, users must implement a function that returns the items in the dataset and then inform detectron2 about this function. For instance, a subset of the labels ["car", "chair", "cup", "person", and "traffic light"] are used for training and detecting errors in this notebook.

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "data/coco/annotations/instances_train2017_5labels.json",
                        "data/coco/train2017/")
register_coco_instances("my_dataset_val", {}, "data/coco/annotations/instances_val2017_5labels.json",
                        "data/coco/val2017/")
from detectron2.evaluation import (
    CityscapesInstanceEvaluator,
    CityscapesSemSegEvaluator,
    COCOEvaluator,
    COCOPanopticEvaluator,
    DatasetEvaluators,
    LVISEvaluator,
    PascalVOCDetectionEvaluator,
    SemSegEvaluator,
    verify_results,
)

We define the configuration settings for training an object detection model using Detectron2. The model architecture used in this example is "faster_rcnn_X_101_32x8d_FPN_3x" from the COCO-Detection model zoo. The training data is specified by the "my_dataset_train" dataset and validation data is specified by the "my_dataset_val" dataset which refer to COCO2017 train and val containing only the subset of labels specified before.

The number of worker threads is set to 2 and the batch size is set to 2.
The learning rate and maximum number of iterations are also specified. The model is initialized from the COCO-Detection model zoo and the output directory for the trained model is created. Finally, the configuration is passed to the DefaultTrainer class for training the object detection model.

<strong>Note:</strong> The number of iterations was set based on early stopping

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 30000    # 
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5  # only 5 classes ["car", "chair", "cup", "person", and "traffic light"] 
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)


In [ ]:
trainer = DefaultTrainer(cfg) 
# trainer.resume_or_load(resume=True)
trainer.train()

## Inference & evaluation using the trained model
If you wish to load a trained model to run inference

In [ ]:
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import cv2
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")


In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

The given code block implements a function "conv" that converts the output of Detectron2 to a format that can be used by cleanlab library for identifying label errors. The function accepts the predicted instances and the number of classes as inputs. It processes the predicted bounding boxes and scores for each instance, and outputs a list of numpy arrays containing the bounding boxes and scores for each class.

In [ ]:
def conv(ins,num_classes):
    fields = ins.get_fields()
    boxes = fields['pred_boxes'].tensor.numpy()
    res = [[] for i in range(num_classes)]
    for i in range(0,len(fields['pred_classes'])):
        pred_class = fields['pred_classes'][i].item()
        probs = ins.get_fields()['scores'][i].item()
        box_cord = list(boxes[i])
        box_cord.append(probs)
        res[pred_class].append(box_cord)
    res2 = []
    for i in res:
        if len(i)==0:
            q = np.array(i,dtype=np.float32).reshape((0,num_classes))
        else:
            q = np.array(i,dtype=np.float32)
        res2.append(q)
    return res2
    return [np.array(i,dtype=np.float32) for i in res]
        

results = []
for i in datas:
    im_name = "data/coco/val2017/"+i['seg_map'].replace(".png",'.jpg')
    im = cv2.imread(im_name)
    outputs = predictor(im)
    results.append(conv(outputs["instances"].to("cpu"),num_classes))
    if len(results)%100==0:
        print(len(results)/len(datas)*100,'% complete')

pickle.dump(results,open("detectron_coco_5labels_inference_more_epochs.pkl",'wb'))